In [1]:
import os
import numpy as np
import pandas as pd
# List of all the datasets that are needed
os.listdir(os.getcwd())

['amazon_transactions.csv',
 '.ipynb_checkpoints',
 'MidtermAlgorithm.ipynb',
 'best_buy_items.csv',
 'amazon_items.csv',
 'best_buy_transactions.csv',
 'k-mart_items.csv',
 'k-mart_transactions.csv',
 'nike_items.csv',
 '.git',
 'nike_transactions.csv']